# Date of observation in monthly median composites

In [ ]:
%matplotlib inline

import datacube
import numpy as np
import sys
import xarray as xr

sys.path.append('../Scripts')
from deafrica_datahandling import load_ard
from deafrica_plotting import rgb

In [ ]:
dc = datacube.Datacube(app='Vegetation_phenology')

In [ ]:
# Define area of interest
lat = 10.3200 
lon = 4.4709
lon_buffer = 0.4
lat_buffer = 0.4

# Combine central lat,lon with buffer to get area of interest
lat_range = (lat-lat_buffer, lat+lat_buffer)
lon_range = (lon-lon_buffer, lon+lon_buffer)

# Set the range of dates for the analysis
years_range = ('2019-01', '2019-12')

In [ ]:
# Create a reusable query
query = {
    'y': lat_range,
    'x': lon_range,
    'time': years_range,
    'measurements': ['red', 'green', 'blue'],
    'resolution': (-10,10),
    'output_crs': 'epsg:32631'
}

# Load available data from Landsat 8
ds = load_ard(dc=dc,
              products=['s2_l2a'],
              min_gooddata=0.99,
              **query,
              dtype='native'
              )

In [ ]:
rgb(ds, index=[0,1,2])

In [ ]:
def date_of_median(da, year, sample_lat, sample_lon):
    """
    da = xr.DataArray
        Assuming an annual time-series
    year = str
        year of time-series in 'da'
    sample_lat = float
        latitude pixel coordinate
    sample_lon = float
        longitude pixel coordinate
    
    """
    
    #calculate medians for each month
    monthly_medians = da.groupby('time.month').median()
    
    months = [str(i) for i in range(1,13)]
    indexes = [i for i in range(0,12)]
    
    dates=[]
    values=[]
    for month, index in zip(months,indexes): 
        
        #select the month of interest from da
        m = da.sel(time=year+"-"+month)
        
        #find regions with all-NaN slices
        mask = m.isnull().all('time')
        
        #calculate distance each pixel has from median
        distance = m - monthly_medians.isel(month=index)
        
        #index of the absolute minimum distance
        distance = distance.fillna(float(distance.max() + 1))
        distance=xr.ufuncs.fabs(distance)
        idx = distance.idxmin(dim='time', skipna=True).where(~mask)
        value = distance.sel(time=idx, method='nearest')
        values.append(value)
        dates.append(idx)
    
    #join into dataarray along new dimension
    dates = xr.concat(dates, "date of median")
    dist_from_median = xr.concat(values, 'dist_from_monthly_median')
    
    #select pixel
    dates = dates.sel(x=sample_lon, y=sample_lat, method='nearest')
    dist_from_median = dist_from_median.sel(x=sample_lon, y=sample_lat, method='nearest')
    
    return dates, dist_from_median

In [ ]:
date_of_median, dist_from_median = date_of_median(ds.blue,
                                                  sample_lon=1929690.,
                                                  sample_lat=-4123870.,
                                                  year='2018')

In [ ]:
print(date_of_median)

In [ ]:
dist_from_median.plot()